In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import src.config as config


In [8]:


import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/325782
Connected. Call `.close()` to terminate connection gracefully.


In [9]:
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)


Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/325782/fs/324677/fv/time_series_hourly_feature_view/version/1


In [10]:
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides',
)


Finished: Reading data from Hopsworks, using Hive (1605.01s) 


In [11]:
ts_data.head()

,pickup_hour,rides,pickup_location_id
0,2022-12-31 06:00:00,0,119
1,2022-02-19 12:00:00,0,243
2,2023-09-03 06:00:00,0,246
3,2022-02-23 00:00:00,1,33
4,2022-03-24 15:00:00,0,171


In [12]:


from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28, # one month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')



  0%|          | 0/265 [00:00<?, ?it/s]DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  0%|          | 1/265 [00:00<00:47,  5.61it/s]DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  1%|          | 2/265 [00:00<00:47,  5.58it/s]DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  1%|          | 3/265 [00:00<00:46,  5.62it/s]DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0

features_and_target.shape=(185235, 675)


In [13]:


from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

# training data -> from January 2022 up until 2 months ago
# test data -> last 2 months
cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*1))

print(f'{cutoff_date=}')

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'   
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')



cutoff_date=Timestamp('2023-11-26 00:00:00')
X_train.shape=(177550, 674)
y_train.shape=(177550,)
X_test.shape=(7685, 674)
y_test.shape=(7685,)


In [14]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [15]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1)

[I 2023-12-24 20:31:13,936] A new study created in memory with name: no-name-c45d822e-c6ff-4715-9bd4-ed776fb7abad
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [16]:


best_params = study.best_trial.params
print(f'{best_params=}')



best_params={'num_leaves': 250, 'feature_fraction': 0.7248576870312278, 'bagging_fraction': 0.684871652619132, 'min_child_samples': 96}


In [17]:


pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)



[LightGBM] [Warning] feature_fraction is set=0.7248576870312278, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7248576870312278
[LightGBM] [Warning] bagging_fraction is set=0.684871652619132, subsample=1.0 will be ignored. Current value: bagging_fraction=0.684871652619132
[LightGBM] [Warning] feature_fraction is set=0.7248576870312278, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7248576870312278
[LightGBM] [Warning] bagging_fraction is set=0.684871652619132, subsample=1.0 will be ignored. Current value: bagging_fraction=0.684871652619132
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.648105 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 171901
[LightGBM] [Info] Number of data points in the train set: 177550, number of used features: 676
[LightGBM] [Info] Start training from score 1

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x7ff16c171af0>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.684871652619132,
                               feature_fraction=0.7248576870312278,
                               min_child_samples=96, num_leaves=250))])

In [18]:


predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')



[LightGBM] [Warning] feature_fraction is set=0.7248576870312278, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7248576870312278
[LightGBM] [Warning] bagging_fraction is set=0.684871652619132, subsample=1.0 will be ignored. Current value: bagging_fraction=0.684871652619132
test_mae=3.5745


In [20]:
import joblib
from src.paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

['/home/jithish/taxi_demand_predictor/models/model.pkl']

In [21]:


from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)



In [24]:


model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor with a bit of hyper-parameter tuning",
    input_example=X_train.sample(),
    model_schema=model_schema
)


model.save(str(MODELS_DIR / 'model.pkl'))



Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/2301158 elapsed<00:00 remaining<?

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


Uploading: 0.000%|          | 0/3393 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/58131 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/325782/models/taxi_demand_predictor_next_hour/1


Model(name: 'taxi_demand_predictor_next_hour', version: 1)